# Napovedovanje primernega športa glede na fizične karakteristike

Ustvarili smo model, ki napove šport za katerega imamo najbolj primerno fizično predpostavko
Za napoved moramo vnesti svojo starost, višino, težo ter spol. Model nam nato vrne tri najbolj primerne športe.


### Ustvarjanje modela

izmed vseh 66 športov, ki jih imamo med podatki, smo odstranili 9 športov, saj so bili le na nekaterih igrah. Športi
z premalo podatkov so vlečenjne vrvi, golf, rugby, umetniško tekmovanje, lacrose itd. Tako model napoveduje med 57 športi. 


### Podatki

Iz naše baze podatkov, smo izluščili vse športnike, ki so dobili medaljo ter nimajo mankajočih podatkov. Vsak športnik je predstavljen s starostjo, višino, težo, spolam ter športam v katerem je osvojil medaljo. Ker je za nekatere športe kot so atletika, košarka veliko več podatkov smo naključno nekatere odstranili ter tako prišli do veliko bolj uravnovešenih podatkov. Nazadnje je sledila nakjlučna razvrstitev podatkov v učno ter testno množico v razmerju 70:30

### Učenje modela in testiranje modela

Ker podatki niso tako zelo dobro uteženi je za klasifikator uporabljen RandomForest, saj od vseh modelov deluje bolje na neuravnovešenih podatkih.

Model prvi šport napove z 30% natančnostjo, če pa vzamemo prve tri najbolj vrjetne športe model napoveduje z več kot 50% natančnostjo.

### Primeri

Moški: 181cm 72kg 21 let
1.   Athletics
2.   Shooting
3.   Figure Skating

Moški: 181cm 72kg 33 let
1.   Canoeing
2.   Diving
3.   Equestrianism(konjeništvo)

Ženska: 181cm 72kg 21 let
1.   Luge
2.   Speed Skating
3.   Shooting

Ženska: 195cm 80kg 24 let
1.   Basketball
2.   Speed Skating
3.   Luge

Ženska: 160cm 60kg 23 let
1.   Synchronized Swimming
2.   Gymnastics
3.   Golf

Moški: 160cm 60kg 23 let
1.   Table Tennis
2.   Handball
3.   Football

In [16]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import math
import sklearn as sk  
from sklearn.linear_model import LogisticRegression  
from sklearn import svm  
from sklearn.ensemble import RandomForestClassifier  
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder

raw_data = pd.read_csv("Data/athlete_events.csv")
raw_regions = pd.read_csv("Data/noc_regions.csv")

In [17]:
all_raw_data = pd.merge(raw_data, raw_regions, on="NOC")

In [18]:
#pridobivanje potrebnih podatkov
datatmp = all_raw_data[['Sport','Age','Height','Weight','Medal','Sex']].copy()
datatmp.dropna()
#brisanje vseh nepopolnih vrstic
datatmp = datatmp[pd.notnull(datatmp['Medal'])]
datatmp = datatmp[pd.notnull(datatmp['Sport'])]
datatmp = datatmp[pd.notnull(datatmp['Weight'])]
datatmp = datatmp[pd.notnull(datatmp['Height'])]
datatmp = datatmp[pd.notnull(datatmp['Age'])]
datatmp = datatmp[pd.notnull(datatmp['Sex'])]
datatmp2 = np.array(datatmp)
del datatmp['Medal']


In [19]:
#spreminjanje stringov v int
le = LabelEncoder()  
datatmp['Sex'] = le.fit_transform(datatmp['Sex'])
lebel = LabelEncoder()  
datatmp['Sport'] = lebel.fit_transform(datatmp['Sport'])

#brisanje starih sportov ki so bili le na nekaterih igrah
datatmp = datatmp.loc[datatmp['Sport'] != 2]
datatmp = datatmp.loc[datatmp['Sport'] != 20]
datatmp = datatmp.loc[datatmp['Sport'] != 50]
datatmp = datatmp.loc[datatmp['Sport'] != 21]
datatmp = datatmp.loc[datatmp['Sport'] != 38]
datatmp = datatmp.loc[datatmp['Sport'] != 48]
datatmp = datatmp.loc[datatmp['Sport'] != 33]
datatmp = datatmp.loc[datatmp['Sport'] != 34]
datatmp = datatmp.loc[datatmp['Sport'] != 27]



In [20]:
#nakljucno razvrscanje podatkov v train in test mnozici
def train_validate_test_split(df, train_percent=.7, validate_percent=.3, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.loc[perm[:train_end]]
    validate = df.loc[perm[train_end:validate_end]]
    return train, validate
train_validate_test_split
train,test = train_validate_test_split(datatmp)

In [21]:
y_tr = train.iloc[:,0]  
X_tr = train.iloc[:,1:]

y_test = test.iloc[:,0]  
X_test = test.iloc[:,1:]

In [22]:
RF = RandomForestClassifier(n_estimators=2000, max_depth=14, random_state=0).fit(X_tr, y_tr)  
RF.predict(X_test)  
print(round(RF.score(X_test, y_test), 4))

0.3068


In [93]:
X_test
moji = [[23,160,60,1]]
predictions_test = RF.predict_proba(moji)
predictions_test = predictions_test[0]

i = 1;
i1 = 0;
i2 = 0;
i3 = 0;
tmp1 = 0;
tmp2 = 0;
tmp3 = 0;
for a in predictions_test:
    if(a > tmp1):
        i1 = i
        tmp1 = a
    i+=1
predictions_test[i1-1]=0
i = 1
for a in predictions_test:
    if(a > tmp2):
        i2 = i
        tmp2 = a
    i+=1
predictions_test[i2-1]=0   
i = 1
for a in predictions_test:
    if(a > tmp3):
        i3 = i
        tmp3 = a
    i+=1

In [94]:
first = lebel.inverse_transform([i1])
print("1."," ",first[0])
second = lebel.inverse_transform([i2])
print("2."," ",second[0])
third = lebel.inverse_transform([i3])
print("3."," ",third[0])

1.   Table Tennis
2.   Handball
3.   Football
